In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/paysim-data/paysim dataset.csv


In [2]:
import numpy as np
import pandas as pd

In [3]:
PATH = "/kaggle/input/paysim-data/paysim dataset.csv" 

# Read the CSV file into a Pandas DataFrame
df = pd.read_csv(PATH)

# Display the some random rows of the DataFrame
df.sample(10)

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
6124247,540,PAYMENT,23688.13,C18599525,64304.00,40615.87,M1054008375,0.00,0.00,0,0
1505671,143,TRANSFER,254853.52,C2065823705,0.00,0.00,C409956115,7516377.36,7771230.88,0,0
595749,33,PAYMENT,7063.05,C512684768,150173.39,143110.35,M958056252,0.00,0.00,0,0
2765211,213,PAYMENT,13571.67,C147413689,88945.00,75373.33,M1901171608,0.00,0.00,0,0
5646730,396,CASH_OUT,71677.03,C1488353176,24668.00,0.00,C1415000082,0.00,71677.03,0,0
3867446,283,CASH_IN,155715.27,C107422891,6680436.70,6836151.98,C1801050179,512463.91,356748.64,0,0
3984847,298,CASH_OUT,143810.17,C862627108,0.00,0.00,C1759735495,194725.81,338535.98,0,0
4517883,325,CASH_OUT,65829.37,C116953877,64851.00,0.00,C994335556,76214.00,142043.37,0,0
6187141,569,TRANSFER,1016126.76,C1369867680,0.00,0.00,C1171803243,11198749.29,12214876.06,0,0
2491238,204,CASH_IN,289029.41,C1785585198,11334660.49,11623689.91,C1179059245,886578.83,597549.41,0,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6362620 entries, 0 to 6362619
Data columns (total 11 columns):
 #   Column          Dtype  
---  ------          -----  
 0   step            int64  
 1   type            object 
 2   amount          float64
 3   nameOrig        object 
 4   oldbalanceOrg   float64
 5   newbalanceOrig  float64
 6   nameDest        object 
 7   oldbalanceDest  float64
 8   newbalanceDest  float64
 9   isFraud         int64  
 10  isFlaggedFraud  int64  
dtypes: float64(5), int64(3), object(3)
memory usage: 534.0+ MB


In [5]:
df.isnull().sum()

step              0
type              0
amount            0
nameOrig          0
oldbalanceOrg     0
newbalanceOrig    0
nameDest          0
oldbalanceDest    0
newbalanceDest    0
isFraud           0
isFlaggedFraud    0
dtype: int64

### `type`,`nameOrig` and `nameDest` are of object datatype and every column does not have null values

In [6]:
df['isFraud'].value_counts()

isFraud
0    6354407
1       8213
Name: count, dtype: int64

### This confirms a highly imbalanced dataset typical in fraud detection scenarios.

> First object type colums

1. `type`
2. `nameOrig`
3. `nameDest`

In [7]:
def print_value_count(df,columnName):
    print(df[columnName].value_counts())
    print()
    print(columnName.upper(),"column has",len(df[columnName].unique()),"unique categories..")

In [8]:
print_value_count(df,'type')

type
CASH_OUT    2237500
PAYMENT     2151495
CASH_IN     1399284
TRANSFER     532909
DEBIT         41432
Name: count, dtype: int64

TYPE column has 5 unique categories..


### Given that there are only 5 unique values, one-hot encoding is a feasible approach for `type` column.
> Now for `nameOrig`


In [9]:
print_value_count(df,'nameOrig')

nameOrig
C1902386530    3
C363736674     3
C545315117     3
C724452879     3
C1784010646    3
              ..
C98968405      1
C720209255     1
C1567523029    1
C644777639     1
C1280323807    1
Name: count, Length: 6353307, dtype: int64

NAMEORIG column has 6353307 unique categories..


### With 6,353,307 unique values in the `nameOrig` column, it is clear that this column likely represents unique identifiers for each customer or account involved in the transactions. Given the large number of unique values, this column can be considered as having high cardinality.

In [10]:
print_value_count(df,'nameDest')

nameDest
C1286084959    113
C985934102     109
C665576141     105
C2083562754    102
C248609774     101
              ... 
M1470027725      1
M1330329251      1
M1784358659      1
M2081431099      1
C2080388513      1
Name: count, Length: 2722362, dtype: int64

NAMEDEST column has 2722362 unique categories..


### With 2,722,362 unique values in the `nameDest` column, it is clear that this column likely represents unique identifiers for each customer or account involved in the transactions. Given the large number of unique values, this column can be considered as having high cardinality.

In [11]:
print("No of rows in data frame:",df.shape [0])

No of rows in data frame: 6362620


In [12]:
df.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


### Approach 1: We should drop high cardinality columns:{`nameOrig`,`nameDest`}

1. Encode the `type` column using one-hot encoding


In [4]:
# One-hot encode the 'type' column
df_encoded = pd.get_dummies(df, columns=['type'])

df_encoded.head()

,step,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud,type_CASH_IN,type_CASH_OUT,type_DEBIT,type_PAYMENT,type_TRANSFER
0,1,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0,False,False,False,True,False
1,1,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0,False,False,False,True,False
2,1,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0,False,False,False,False,True
3,1,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0,False,True,False,False,False
4,1,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0,False,False,False,True,False


2. Drop `nameOrig` and `nameDest` columns (since they have too many unique categories):

In [15]:
df_encoded = df_encoded.drop(columns=['nameOrig', 'nameDest'])

3. Scale the numerical features:

In [16]:
from sklearn.preprocessing import StandardScaler

# Select numerical columns to scale
numerical_cols = ['amount', 'oldbalanceOrg', 'newbalanceOrig', 'oldbalanceDest', 'newbalanceDest']

scaler = StandardScaler()
df_encoded[numerical_cols] = scaler.fit_transform(df_encoded[numerical_cols])

df_encoded.head()

,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud,type_CASH_IN,type_CASH_OUT,type_DEBIT,type_PAYMENT,type_TRANSFER
0,1,-0.281560,-0.229810,-0.237622,-0.323814,-0.333411,0,0,False,False,False,True,False
1,1,-0.294767,-0.281359,-0.285812,-0.323814,-0.333411,0,0,False,False,False,True,False
2,1,-0.297555,-0.288654,-0.292442,-0.323814,-0.333411,1,0,False,False,False,False,True
3,1,-0.297555,-0.288654,-0.292442,-0.317582,-0.333411,1,0,False,True,False,False,False
4,1,-0.278532,-0.274329,-0.282221,-0.323814,-0.333411,0,0,False,False,False,True,False


4. Split the dataset into training and testing sets:

In [17]:
from sklearn.model_selection import train_test_split

# Define features and target
X = df_encoded.drop(columns=['isFraud', 'isFlaggedFraud'])
y = df_encoded['isFraud']

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(5090096, 11) (1272524, 11) (5090096,) (1272524,)


5. Train a Random Forest Classifier:

In [18]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Initialize the Random Forest model
rf_model = RandomForestClassifier(random_state=42, n_jobs=-1, class_weight='balanced')

# Train the model
rf_model.fit(X_train, y_train)

RandomForestClassifier(class_weight='balanced', n_jobs=-1, random_state=42)

6. Evaluate the Model:

In [19]:
# Make predictions
y_pred = rf_model.predict(X_test)
y_proba = rf_model.predict_proba(X_test)[:, 1]

# Calculate evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_proba)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"ROC-AUC: {roc_auc:.4f}")

Accuracy: 0.9997
Precision: 0.9860
Recall: 0.7840
F1 Score: 0.8735
ROC-AUC: 0.9943


#### **Handling Imbalanced Data:** Since recall is slightly lower, we must try SMOTE (Synthetic Minority Over-sampling Technique) to improve recall with class weights (class importance).

> **Class Weights:** Adjusts the importance of each class in the loss function. It doesn't change the data distribution but helps the model focus more on the minority class.
> **SMOTE:** Generates synthetic samples to balance the dataset, leading to a more balanced class distribution in the training set.

> Class weights is only used..

In [22]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Define features and target
X = df_encoded.drop(columns=['isFraud', 'isFlaggedFraud'])
y = df_encoded['isFraud']

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Manually set class weights (e.g., class 1 is 12 times more important)
class_weight = {0: 1, 1: 12}

# Initialize the Random Forest model with manual class weights
rf_model = RandomForestClassifier(random_state=42, n_jobs=-1, class_weight=class_weight)

# Train the model
rf_model.fit(X_train, y_train)

# Evaluate the model
y_pred = rf_model.predict(X_test)
y_proba = rf_model.predict_proba(X_test)[:, 1]

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_proba)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"ROC-AUC: {roc_auc:.4f}")

Accuracy: 0.9997
Precision: 0.9832
Recall: 0.7969
F1 Score: 0.8803
ROC-AUC: 0.9943


#### When comparing the scores from using class weights only versus a vanilla Random Forest model, 

It is evident that using class weights improves recall at the expense of a slight drop in precision. 
This results in a higher F1 score, indicating a better balance between precision and recall.



#### Observations:

1. **Accuracy:** Remains unchanged, which is expected since the dataset is highly imbalanced, and accuracy is dominated by the majority class.
2. **Precision:** Slightly decreased from 0.9860 to 0.9832.
3. **Recall:** Improved from 0.7840 to 0.7969, indicating the model is better at identifying fraudulent transactions.
4. **F1 Score:** Increased from 0.8735 to 0.8803, showing an overall improvement in the model's performance.
5. **ROC-AUC:** Remains the same at 0.9943, indicating the model's ability to distinguish between classes is unchanged.

-----

> Combining SMOTE with Class Weights


In [23]:
from imblearn.over_sampling import SMOTE

# Apply SMote to the training set
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Initialize the Random Forest model with class weights
class_weight = {0: 1, 1: 12}
rf_model = RandomForestClassifier(random_state=42, n_jobs=-1, class_weight=class_weight)

# Train the model
rf_model.fit(X_train_resampled, y_train_resampled)

# Evaluate the model
y_pred = rf_model.predict(X_test)
y_proba = rf_model.predict_proba(X_test)[:, 1]

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_proba)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"ROC-AUC: {roc_auc:.4f}")

Accuracy: 0.9988
Precision: 0.5204
Recall: 0.9778
F1 Score: 0.6792
ROC-AUC: 0.9986


-----

### Approach 2: Hash high cardinality columns:{`nameOrig`,`nameDest`}

> Hashing along with class weights without SMOTE

In [26]:
from sklearn.feature_extraction import FeatureHasher
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import pandas as pd

# Define features and target
X = df.drop(columns=['isFraud', 'isFlaggedFraud'])
y = df['isFraud']

# One-hot encode the 'type' column
X = pd.get_dummies(X, columns=['type'], drop_first=True)

# Use FeatureHasher for hashing the 'nameOrig' and 'nameDest' columns
hasher = FeatureHasher(n_features=10, input_type='string')

# Transform the columns
hashed_nameOrig = hasher.transform(X['nameOrig'].astype(str).apply(lambda x: [x])).toarray()
hashed_nameDest = hasher.transform(X['nameDest'].astype(str).apply(lambda x: [x])).toarray()

# Convert hashed arrays to DataFrame
hashed_nameOrig_df = pd.DataFrame(hashed_nameOrig, columns=[f'nameOrig_{i}' for i in range(hashed_nameOrig.shape[1])])
hashed_nameDest_df = pd.DataFrame(hashed_nameDest, columns=[f'nameDest_{i}' for i in range(hashed_nameDest.shape[1])])

# Drop the original 'nameOrig' and 'nameDest' columns and concatenate the hashed DataFrames
X = X.drop(columns=['nameOrig', 'nameDest'])
X = pd.concat([X.reset_index(drop=True), hashed_nameOrig_df, hashed_nameDest_df], axis=1)

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the Random Forest model
rf_model = RandomForestClassifier(random_state=42, n_jobs=-1, class_weight={0: 1, 1: 12})

# Train the model
rf_model.fit(X_train, y_train)

# Evaluate the model
y_pred = rf_model.predict(X_test)
y_proba = rf_model.predict_proba(X_test)[:, 1]

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_proba)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"ROC-AUC: {roc_auc:.4f}")

Accuracy: 0.9997
Precision: 0.9919
Recall: 0.7586
F1 Score: 0.8597
ROC-AUC: 0.9942


#### Observations:

1. **Accuracy: 0.9997**: This high accuracy suggests that the model is generally performing well, but accuracy alone can be misleading, especially in imbalanced datasets.
2. **Precision: 0.9919:** This indicates that when the model predicts a transaction as fraudulent, it is correct about 99% of the time. This is very good and suggests that the model is not flagging too many legitimate transactions as fraud.
3. **Recall: 0.7586:** This value shows that the model is identifying about 76% of actual fraudulent transactions. While decent, there is still room for improvement in capturing more fraud cases.
4. **F1 Score: 0.8597:** This score balances precision and recall, showing a good trade-off between the two metrics. An F1 score above 0.8 is typically considered strong in many contexts.
5. **ROC-AUC: 0.9942:** This indicates excellent model performance, as the model can distinguish between classes very effectively across different thresholds.

> Hashing along with class weights and SMOTE code

In [5]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
from sklearn.feature_extraction import FeatureHasher
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import OneHotEncoder

# Define features and target
X = df.drop(columns=['isFraud', 'isFlaggedFraud'])
y = df['isFraud']

# One-hot encode the 'type' column
ohe = OneHotEncoder(sparse_output=False)
type_encoded = ohe.fit_transform(X[['type']])

# Convert encoded arrays to DataFrame
type_encoded_df = pd.DataFrame(type_encoded, columns=ohe.get_feature_names_out(['type']))

# Use FeatureHasher for hashing the 'nameOrig' and 'nameDest' columns
hasher = FeatureHasher(n_features=10, input_type='string')
hashed_nameOrig = hasher.transform(X['nameOrig'].apply(lambda x: [x]).tolist()).toarray()
hashed_nameDest = hasher.transform(X['nameDest'].apply(lambda x: [x]).tolist()).toarray()

# Convert hashed arrays to DataFrame
hashed_nameOrig_df = pd.DataFrame(hashed_nameOrig, columns=[f'nameOrig_{i}' for i in range(hashed_nameOrig.shape[1])])
hashed_nameDest_df = pd.DataFrame(hashed_nameDest, columns=[f'nameDest_{i}' for i in range(hashed_nameDest.shape[1])])

# Drop the original 'type', 'nameOrig', and 'nameDest' columns and concatenate the encoded and hashed DataFrames
X = X.drop(columns=['type', 'nameOrig', 'nameDest'])
X = pd.concat([X.reset_index(drop=True), type_encoded_df.reset_index(drop=True), hashed_nameOrig_df, hashed_nameDest_df], axis=1)

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Apply SMOTE to the training set
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

# Initialize the Random Forest model with class weights
rf_model = RandomForestClassifier(random_state=42, n_jobs=-1, class_weight={0: 1, 1: 12})

# Train the model
rf_model.fit(X_train_res, y_train_res)

# Evaluate the model
y_pred = rf_model.predict(X_test)
y_proba = rf_model.predict_proba(X_test)[:, 1]

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_proba)
conf_matrix = confusion_matrix(y_test, y_pred)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"ROC-AUC: {roc_auc:.4f}")
print(f"Confusion Matrix:\n{conf_matrix}")

Accuracy: 0.9996
Precision: 0.9895
Recall: 0.6994
F1 Score: 0.8195
ROC-AUC: 0.9906
Confusion Matrix:
[[1270892      12]
 [    487    1133]]


### Trying different models

> Logistic Regression model (Has faster training time)

In [7]:
import pandas as pd
from sklearn.feature_extraction import FeatureHasher
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Define features and target
X = df.drop(columns=['isFraud', 'isFlaggedFraud'])
y = df['isFraud']

# Use FeatureHasher for hashing the 'nameOrig' and 'nameDest' columns
hasher = FeatureHasher(n_features=10, input_type='string')
hashed_nameOrig = hasher.transform(X['nameOrig'].astype(str).apply(lambda x: [x])).toarray()
hashed_nameDest = hasher.transform(X['nameDest'].astype(str).apply(lambda x: [x])).toarray()

# Convert hashed arrays to DataFrame
hashed_nameOrig_df = pd.DataFrame(hashed_nameOrig, columns=[f'nameOrig_{i}' for i in range(hashed_nameOrig.shape[1])])
hashed_nameDest_df = pd.DataFrame(hashed_nameDest, columns=[f'nameDest_{i}' for i in range(hashed_nameDest.shape[1])])

# Drop the original 'nameOrig' and 'nameDest' columns and concatenate the hashed DataFrames
X = X.drop(columns=['nameOrig', 'nameDest'])
X = pd.concat([X.reset_index(drop=True), hashed_nameOrig_df, hashed_nameDest_df], axis=1)

# One-hot encode the 'type' column
X = pd.get_dummies(X, columns=['type'], drop_first=True)

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the Logistic Regression model
log_reg = LogisticRegression(max_iter=1000, class_weight={0: 1, 1: 12}, n_jobs=-1, random_state=42)

# Train the model
log_reg.fit(X_train, y_train)

# Evaluate the model
y_pred = log_reg.predict(X_test)
y_proba = log_reg.predict_proba(X_test)[:, 1]

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_proba)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"ROC-AUC: {roc_auc:.4f}")


Accuracy: 0.9937
Precision: 0.1157
Recall: 0.5994
F1 Score: 0.1939
ROC-AUC: 0.9076


The results from the Logistic Regression model show that it did not perform as well as the Random Forest models in terms of precision, recall, F1 score, and ROC-AUC. Given the context of your problem and the results from various models, it appears that Random Forest is more suitable for your dataset despite its longer training time.

> XGBoost (with SMOTE and hashing)

In [8]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from imblearn.over_sampling import SMOTE

# Define features and target
X = df.drop(columns=['isFraud', 'isFlaggedFraud'])
y = df['isFraud']

# Use FeatureHasher for hashing the 'nameOrig' and 'nameDest' columns
hasher = FeatureHasher(n_features=10, input_type='string')
hashed_nameOrig = hasher.transform(X['nameOrig'].astype(str).apply(lambda x: [x])).toarray()
hashed_nameDest = hasher.transform(X['nameDest'].astype(str).apply(lambda x: [x])).toarray()

# Convert hashed arrays to DataFrame
hashed_nameOrig_df = pd.DataFrame(hashed_nameOrig, columns=[f'nameOrig_{i}' for i in range(hashed_nameOrig.shape[1])])
hashed_nameDest_df = pd.DataFrame(hashed_nameDest, columns=[f'nameDest_{i}' for i in range(hashed_nameDest.shape[1])])

# Drop the original 'nameOrig' and 'nameDest' columns and concatenate the hashed DataFrames
X = X.drop(columns=['nameOrig', 'nameDest'])
X = pd.concat([X.reset_index(drop=True), hashed_nameOrig_df, hashed_nameDest_df], axis=1)

# One-hot encode the 'type' column
X = pd.get_dummies(X, columns=['type'], drop_first=True)

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Apply SMOTE to handle class imbalance
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

# Initialize the XGBoost model
xgb_model = xgb.XGBClassifier(random_state=42, scale_pos_weight=(len(y_train) - sum(y_train)) / sum(y_train))

# Train the model
xgb_model.fit(X_train_smote, y_train_smote)

# Evaluate the model
y_pred = xgb_model.predict(X_test)
y_proba = xgb_model.predict_proba(X_test)[:, 1]

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_proba)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"ROC-AUC: {roc_auc:.4f}")


Accuracy: 0.8021
Precision: 0.0063
Recall: 0.9809
F1 Score: 0.0125
ROC-AUC: 0.8792


The XGBoost model, in this case, has a high recall but very low precision, which means it correctly identifies most fraudulent transactions but also has a high number of false positives. This is not ideal for fraud detection, where precision is crucial to minimize the cost of investigating false positives.

> LightGBM

In [9]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from imblearn.over_sampling import SMOTE

# Define features and target
X = df.drop(columns=['isFraud', 'isFlaggedFraud'])
y = df['isFraud']

# Use FeatureHasher for hashing the 'nameOrig' and 'nameDest' columns
hasher = FeatureHasher(n_features=10, input_type='string')
hashed_nameOrig = hasher.transform(X['nameOrig'].astype(str).apply(lambda x: [x])).toarray()
hashed_nameDest = hasher.transform(X['nameDest'].astype(str).apply(lambda x: [x])).toarray()

# Convert hashed arrays to DataFrame
hashed_nameOrig_df = pd.DataFrame(hashed_nameOrig, columns=[f'nameOrig_{i}' for i in range(hashed_nameOrig.shape[1])])
hashed_nameDest_df = pd.DataFrame(hashed_nameDest, columns=[f'nameDest_{i}' for i in range(hashed_nameDest.shape[1])])

# Drop the original 'nameOrig' and 'nameDest' columns and concatenate the hashed DataFrames
X = X.drop(columns=['nameOrig', 'nameDest'])
X = pd.concat([X.reset_index(drop=True), hashed_nameOrig_df, hashed_nameDest_df], axis=1)

# One-hot encode the 'type' column
X = pd.get_dummies(X, columns=['type'], drop_first=True)

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Apply SMOTE to handle class imbalance
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

# Initialize the LightGBM model
lgb_model = lgb.LGBMClassifier(random_state=42, scale_pos_weight=(len(y_train) - sum(y_train)) / sum(y_train))

# Train the model
lgb_model.fit(X_train_smote, y_train_smote)

# Evaluate the model
y_pred = lgb_model.predict(X_test)
y_proba = lgb_model.predict_proba(X_test)[:, 1]

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_proba)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"ROC-AUC: {roc_auc:.4f}")


[LightGBM] [Info] Number of positive: 5083503, number of negative: 5083503
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 2.851040 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 6638
[LightGBM] [Info] Number of data points in the train set: 10167006, number of used features: 30
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
Accuracy: 0.9891
Precision: 0.1039
Recall: 0.9938
F1 Score: 0.1882
ROC-AUC: 0.9960


## Tabular Summary and Conclusion

| Model                                   | Accuracy | Precision | Recall | F1 Score | ROC-AUC |
|-----------------------------------------|----------|-----------|--------|----------|---------|
| Vanilla RF Model                       | 0.9997   | 0.9860    | 0.7840 | 0.8735   | 0.9943  |
| RF with Class Weights {0: 1, 1: 12}   | 0.9997   | 0.9832    | 0.7969 | 0.8803   | 0.9943  |
| Combining SMOTE with Class Weights     | 0.9988   | 0.5204    | 0.9778 | 0.6792   | 0.9986  |
| Hashing with Class Weights             | 0.9997   | 0.9919    | 0.7586 | 0.8597   | 0.9942  |
| Hashing + Class Weights + SMOTE        | 0.9996   | 0.9895    | 0.6994 | 0.8195   | 0.9906  |
| Logistic Regression                     | 0.9937   | 0.1157    | 0.5994 | 0.1939   | 0.9076  |
| XGBoost (with SMOTE and hashing)      | 0.8021   | 0.0063    | 0.9809 | 0.0125   | 0.8792  |
| LightGBM                               | 0.9891   | 0.1039    | 0.9938 | 0.1882   | 0.9960  |


### **Conclusion:**

From the table, the best model to use for PaySim fraud detection appears to be **RF with Class Weights {0: 1, 1: 12}**. This model provides a solid balance of precision and recall while maintaining very high accuracy and ROC-AUC scores. Here’s why:

#### **Performance Metrics:**

- **Accuracy:** 0.9997 (very high)
- **Precision:** 0.9832 (high)
- **Recall:** 0.7969 (better than vanilla RF)
- **F1 Score:** 0.8803 (highest among the models)
- **ROC-AUC:** 0.9943 (very high)

This model shows a balanced performance with a notable improvement in recall compared to the vanilla RF model, allowing it to better detect fraudulent transactions without sacrificing much precision. The high F1 score indicates an effective balance between precision and recall, making it a strong candidate for the PaySim fraud detection task.